In [1]:
# Jupyter notebook for testing deepModel library
# author: Fabrizio Romanelli
# email : fabrizio.romanelli@gmail.com
# date  : 04/10/2023

# Import the deepModel library
import deepModel as dm

# Initialize the environment
dm.initialize(CPU=20, GPU=1, VERBOSE='2', NPARRAYS=True)

2023-10-10 10:34:56.147165: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-10 10:34:56.235355: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-10 10:34:58.017035: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-10 10:34:58.021303: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open f

In [2]:
# Multivariate DNN dev test

import tensorflow as tf
from keras import backend as K
from keras.losses import MeanSquaredError

def mse_pearson_loss(y_true, y_pred):
  x = y_true
  y = y_pred
  mx = K.mean(x)
  my = K.mean(y)
  xm, ym = x-mx, y-my
  r_num = K.sum(tf.multiply(xm,ym))
  r_den = K.sqrt(tf.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
  r = r_num / r_den
  r = K.maximum(K.minimum(r, 1.0), -1.0)
  r = K.square(r)
  mse = MeanSquaredError()
  return 1 - r + mse(y_true, y_pred)

mDnn = dm.multivariateDNN(name="multivariate DNN test", inputN=2)
mDnn.setInputs([{'shape': (2,), 'name': 'gino'}, {'shape': (2,), 'name': 'pino'}])

innerLayers = [[{'units': 16, 'activation': 'elu'}, {'units': 16, 'activation': 'elu'}, {'units': 16, 'activation': 'elu'}, {'units': 3, 'activation': 'linear'}]]
innerLayers.append([{'units': 16, 'activation': 'elu'}, {'units': 16, 'activation': 'elu'}, {'units': 16, 'activation': 'elu'}, {'units': 3, 'activation': 'linear'}])
innerLayers.append([{'units': 16, 'activation': 'elu'}, {'units': 16, 'activation': 'elu'}, {'units': 16, 'activation': 'elu'}, {'units': 3, 'activation': 'linear'}])

mDnn.setLayers(innerLayers)

outputLayers = [{'units': 32, 'activation': 'elu'}, {'units': 64, 'activation': 'elu'}, {'units': 128, 'activation': 'elu'}, {'units': 64, 'activation': 'elu'}, {'units': 32, 'activation': 'elu'}, {'units': 1, 'activation': 'linear'}]

mDnn.setOutLayers(outputLayers)
mDnn.setModelConfiguration(optimizer='adam', loss='mse')
# mDnn.setModelConfiguration(optimizer='adam', loss=mse_pearson_loss)

mDnn.build()
# mDnn.summary()

import numpy as np

x1 = np.array([0.2,0.3,0.5,0.6,0.7])
x2 = np.array([1.1,1.2,1.4,1.5,1.6])
X1 = np.array([x1,x2]).T
x3 = np.array([0.2,0.3,0.5,0.6,0.7])
x4 = np.array([1.1,1.2,1.4,1.5,1.6])
X2 = np.array([x3,x4]).T
y  = np.array([3.0,4.0,6.0,7.0,8.0])

mDnn.fit(x=[X1,X2], y=y, epochs=50, shuffle=True, verbose=1)

2023-10-10 10:34:58.625385: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-10 10:34:58.625487: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-10 10:34:58.625503: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-10 10:34:58.625695: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-10 10:34:58.625712: I tensorflow/compile

[DM] Building model...
[DM] Model built!
Epoch 1/50


2023-10-10 10:35:00.496019: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 [==============================] - 3s 3s/step - loss: 32.6453
Epoch 2/50
1/1 [==============================] - 0s 8ms/step - loss: 27.1064
Epoch 3/50
1/1 [==============================] - 0s 10ms/step - loss: 22.0358
Epoch 4/50
1/1 [==============================] - 0s 7ms/step - loss: 17.3620
Epoch 5/50
1/1 [==============================] - 0s 8ms/step - loss: 13.1391
Epoch 6/50
1/1 [==============================] - 0s 12ms/step - loss: 9.4620
Epoch 7/50
1/1 [==============================] - 0s 8ms/step - loss: 6.4265
Epoch 8/50
1/1 [==============================] - 0s 10ms/step - loss: 4.1445
Epoch 9/50
1/1 [==============================] - 0s 11ms/step - loss: 2.7262
Epoch 10/50
1/1 [==============================] - 0s 8ms/step - loss: 2.2260
Epoch 11/50
1/1 [==============================] - 0s 11ms/step - loss: 2.5636
Epoch 12/50
1/1 [==============================] - 0s 10ms/step - loss: 3.4339
Epoch 13/50
1/1 [==============================] - 0s 9ms/step - loss: 4.3

In [3]:
x1 = np.array([2.4])
x2 = np.array([3.3])
X1 = np.array([x1,x2]).T
x3 = np.array([2.4])
x4 = np.array([3.3])
X2 = np.array([x3,x4]).T
y = mDnn.predict([X1,X2])
print(y.numpy())

[[8.210758]]


In [4]:
mDnn.save('test',tflite=False)

[DM] Saving model...
[DM] Model saved!


In [5]:
mDnn2 = dm.multivariateDNN(name="multivariate DNN test 2",inputN=2)
mDnn2.load("test")

[DM] Loading model...
[DM] Loaded!


In [7]:
x1 = np.array([2.4])
x2 = np.array([3.3])
X1 = np.array([x1,x2]).T
x3 = np.array([2.4])
x4 = np.array([3.3])
X2 = np.array([x3,x4]).T
y = mDnn2.predict(x=[X1,X2])
print(y.numpy())

[[8.210758]]
